# Sync Link
### Building the Dataset - Adding Lyrics.com (Writer/Publisher) Info

The next layer to add is writers and publishers via info from Lyrics.com. This service does not have an API but is scrapable.

In [1]:
import pandas as pd
import numpy as np
import requests

In [2]:
sync = pd.read_csv('./data/synced_writer_pub.csv')

In [3]:
sync.head()

,title,artist,year,explicit,styles,languages,title_artist,synced,d_id,d_song,...,d_artist,d_album_id,d_album,d_art,lyric_url,l_title,l_artist,l_album,l_writer,l_pub
0,Tennessee Whiskey,Chris Stapleton,2015,0,"Blues,Rock,Country",English,tennessee whiskey - chris stapleton,1,98975170,Tennessee Whiskey,...,Chris Stapleton,10127538,Traveller,https://cdns-images.dzcdn.net/images/cover/1dd...,/c/chris+stapleton/tennessee+whiskey_21104107....,\nAbout\nTennessee Whiskey lyrics\n,Chris Stapleton,\n from album:\n ...,"Songwriters: Linda H Bartholomew, Dean Dillon",\n Tennessee Whiskey lyrics...
1,Dance Monkey,Tones and I,2019,0,Pop,English,dance monkey - tones and i,1,739870792,Dance Monkey,...,Tones and I,108770322,The Kids Are Coming,https://cdns-images.dzcdn.net/images/cover/563...,/t/tones+and+i/dance+monkey_1711431.html,\nTones And I – Dance Monkey Lyrics,Tones And I,0,Songwriters: Toni Watson,\n Dance Monkey lyrics © Wa...
2,Sweet Caroline,Neil Diamond,1969,0,Pop,English,sweet caroline - neil diamond,1,145434430,Sweet Caroline,...,Neil Diamond,15802430,50th Anniversary Collection,https://cdns-images.dzcdn.net/images/cover/40b...,/n/neil+diamond/sweet+caroline_20098802.html,\nAbout\nSweet Caroline lyrics\n,Neil Diamond,\n from album:\n ...,Songwriters: Neil Diamond,\n Sweet Caroline lyrics © ...
3,Someone You Loved,Lewis Capaldi,2018,0,Pop,English,someone you loved - lewis capaldi,1,582143242,Someone You Loved,...,Lewis Capaldi,78031872,Breach,https://cdns-images.dzcdn.net/images/cover/9a4...,/l/lewis+capaldi/someone+you+loved_21585657.html,\nLewis Capaldi – Someone You Loved Lyrics,Lewis Capaldi,0,"Songwriters: Benjamin Kohn, Lewis Capaldi, Pet...",\n Someone You Loved lyrics...
4,Amazing Grace,Traditional,1831,0,"Traditionnal,Gospel,Blues",English,amazing grace - traditional,1,958704262,Amazing Grace,...,Traditional,147840742,8 Angels Blessing of Song,https://e-cdns-images.dzcdn.net/images/cover/a...,NaN,0,0,0,0,0


In [4]:
sync.isna().sum()

title              0
artist             0
year               0
explicit           0
styles             0
languages          0
title_artist       0
synced             0
d_id               0
d_song             0
d_isrc             0
d_release          0
d_explicit         0
d_bpm              0
d_artist           0
d_album_id         0
d_album            0
d_art              1
lyric_url       3128
l_title            0
l_artist           0
l_album            0
l_writer           0
l_pub              0
dtype: int64

In [10]:
sync[sync['lyric_url'].isna()]['synced'].value_counts()

0    2720
1     408
Name: synced, dtype: int64

In [11]:
sync[sync['lyric_url'].notna()]['synced'].value_counts()

0    7280
1    4283
Name: synced, dtype: int64

In [21]:
sync = sync[(sync['lyric_url'].notna()) & (sync['d_isrc'] != '0') | (sync['artist'] == 'Traditional')]

In [14]:
#Help with Genius API from here: https://stackoverflow.com/questions/47400466/using-genius-api
genius = 'qAc4FAll2im6cOifd-eIc72y6s5883Rg6VLcsLLDLyZIMxK90PAAhHV3UQlbtUzB'

In [33]:
header = {'Authorization' : f'Bearer {genius}'}

In [ ]:
def get_genius(search):
    res = requests.get('https://api.genius.com/search?q=' + search, headers = header)

In [65]:
search = 'someone you loved - lewis capaldi'

In [66]:
res = requests.get('https://api.genius.com/search?q=' + search, headers = header)

In [67]:
res.status_code

200

In [70]:
song = res.json()['response']['hits'][0]['result']['api_path']

In [71]:
res2 = requests.get('https://api.genius.com' + song, headers = header)

In [72]:
res2.status_code

200

In [1]:
info = res2.json()
info

NameError: name 'res2' is not defined

In [95]:
apl_id = info['response']['song']['apple_music_id']
date = info['response']['song']['release_date']
title = info['response']['song']['title']
artist = info['response']['song']['album']['artist']['name']
writers = [x['name'] for x in info['response']['song']['writer_artists']]
album = info['response']['song']['album']['name']

In [107]:
album = info['response']['song']['custom_performances']

In [113]:
album

[{'label': 'Label',
  'artists': [{'api_path': '/artists/649294',
    'header_image_url': 'https://images.genius.com/7b9c33d6905bad2ae8556178667abcdc.900x900x1.png',
    'id': 649294,
    'image_url': 'https://images.genius.com/402247ed5f42660a2c388f18e29caa2f.900x900x1.png',
    'is_meme_verified': False,
    'is_verified': False,
    'name': 'Virgin Records',
    'url': 'https://genius.com/artists/Virgin-records'},
   {'api_path': '/artists/647615',
    'header_image_url': 'https://images.genius.com/f03cc67ee5828928166f5d234dedff40.1000x563x1.jpg',
    'id': 647615,
    'image_url': 'https://images.genius.com/d556ebe6396046228f1a6e326c05b3f3.1000x1000x1.png',
    'is_meme_verified': False,
    'is_verified': False,
    'name': 'Capitol Records',
    'url': 'https://genius.com/artists/Capitol-records'}]},
 {'label': 'Copyright ©',
  'artists': [{'api_path': '/artists/1104563',
    'header_image_url': 'https://images.genius.com/6ee5d6766689e04358c1d0f8ea0ed8f0.600x600x1.png',
    'id':

In [106]:
album[0]['artists'][0]['name']

'Virgin Records'